## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.43,75,95,10.83,overcast clouds
1,1,Georgetown,MY,5.4112,100.3354,82.89,86,20,5.01,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,78.04,70,100,7.25,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,66.47,69,100,5.59,light rain
4,4,Mookane,BW,-23.6880,26.6595,76.21,54,8,5.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,78.04,70,100,7.25,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,66.47,69,100,5.59,light rain
4,4,Mookane,BW,-23.6880,26.6595,76.21,54,8,5.95,clear sky
7,7,Katherine,AU,-14.4667,132.2667,77.32,78,66,8.05,broken clouds
8,8,Victoria,HK,22.2855,114.1577,61.25,90,100,5.01,overcast clouds
11,11,Bredasdorp,ZA,-34.5322,20.0403,69.28,73,1,2.86,clear sky
17,17,Coquimbo,CL,-29.9533,-71.3436,67.44,74,99,5.06,overcast clouds
19,19,Saldanha,ZA,-33.0117,17.9442,69.85,95,22,8.34,few clouds
21,21,Cidreira,BR,-30.1811,-50.2056,75.52,64,20,9.82,few clouds
22,22,Sao Filipe,CV,14.8961,-24.4956,76.17,70,100,5.88,overcast clouds


In [5]:
#Count total rows
preferred_cities_df.notnull().sum()

City_ID                249
City                   249
Country                243
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.replace('', np.nan)

preferred_cities_df = preferred_cities_df.dropna(how='any')

preferred_cities_df.notnull().sum()


City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.04,overcast clouds,-23.1203,-134.9692,
3,Mataura,NZ,66.47,light rain,-46.1927,168.8643,
4,Mookane,BW,76.21,clear sky,-23.6880,26.6595,
7,Katherine,AU,77.32,broken clouds,-14.4667,132.2667,
8,Victoria,HK,61.25,overcast clouds,22.2855,114.1577,
11,Bredasdorp,ZA,69.28,clear sky,-34.5322,20.0403,
17,Coquimbo,CL,67.44,overcast clouds,-29.9533,-71.3436,
19,Saldanha,ZA,69.85,few clouds,-33.0117,17.9442,
21,Cidreira,BR,75.52,few clouds,-30.1811,-50.2056,
22,Sao Filipe,CV,76.17,overcast clouds,14.8961,-24.4956,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
#Visualize DF
hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.04,overcast clouds,-23.1203,-134.9692,People ThankYou
3,Mataura,NZ,66.47,light rain,-46.1927,168.8643,Ellie's Villa
4,Mookane,BW,76.21,clear sky,-23.6880,26.6595,
7,Katherine,AU,77.32,broken clouds,-14.4667,132.2667,Pine Tree Motel
8,Victoria,HK,61.25,overcast clouds,22.2855,114.1577,Mini Hotel Central
11,Bredasdorp,ZA,69.28,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
17,Coquimbo,CL,67.44,overcast clouds,-29.9533,-71.3436,Hostel Del Puerto
19,Saldanha,ZA,69.85,few clouds,-33.0117,17.9442,Blue Bay Lodge
21,Cidreira,BR,75.52,few clouds,-30.1811,-50.2056,Hotel Castelo
22,Sao Filipe,CV,76.17,overcast clouds,14.8961,-24.4956,Hotel Xaguate


In [11]:
#sum rows
hotel_df.notnull().sum()

City                   243
Country                243
Max Temp               243
Current Description    243
Lat                    243
Lng                    243
Hotel Name             243
dtype: int64

In [12]:
#Replace blanks w/ NaN
hotel_df = hotel_df.replace('', np.nan)

In [13]:
#Count Nulls
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             20
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')


In [15]:
#count rows 
clean_hotel_df.notnull().sum()

City                   223
Country                223
Max Temp               223
Current Description    223
Lat                    223
Lng                    223
Hotel Name             223
dtype: int64

In [16]:
#display df
clean_hotel_df.head(25)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,78.04,overcast clouds,-23.1203,-134.9692,People ThankYou
3,Mataura,NZ,66.47,light rain,-46.1927,168.8643,Ellie's Villa
7,Katherine,AU,77.32,broken clouds,-14.4667,132.2667,Pine Tree Motel
8,Victoria,HK,61.25,overcast clouds,22.2855,114.1577,Mini Hotel Central
11,Bredasdorp,ZA,69.28,clear sky,-34.5322,20.0403,Bredasdorp Country Manor
17,Coquimbo,CL,67.44,overcast clouds,-29.9533,-71.3436,Hostel Del Puerto
19,Saldanha,ZA,69.85,few clouds,-33.0117,17.9442,Blue Bay Lodge
21,Cidreira,BR,75.52,few clouds,-30.1811,-50.2056,Hotel Castelo
22,Sao Filipe,CV,76.17,overcast clouds,14.8961,-24.4956,Hotel Xaguate
25,Port Alfred,ZA,77.11,light rain,-33.5906,26.8910,The Halyards Hotel


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
#Create Figure 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))